Data Science Summer School - Split '16
======================================
# Linear and Logistic Regression

Stjepan Begušić

version 0.1.1

`kernel: Python 2.7`

## Introduction

Though it may seem somewhat dull compared to some of the more modern statistical learning approaches seen within the scope of this summer school, linear and logistic regression are still extremely useful and widely used as a statistical learning method. Consequently, a profound understanding of linear methods is an essential weapon in the arsenal of any modern data professional. In this notebook we will address the practical details and implementaion specifics concerning linear classification and regression methods.

Our analysis will rely on implementations of linear methods within the `scikit-learn` package, and will be applied to a dataset containing peer-to-peer lending data on borrowers and investors. Official scikit-learn documentation provides a useful user guide on linear models [[1]](#scikit-linear). The statsmodels package will also be considered for statistical computations and significance testing [[2]](#statsmodels). Instructions on how to install this module (in case it did not come with your Anaconda distribution) can be found here: https://conda.anaconda.org/statsmodels

For additional reading, a very transparent and straightforward representation of these topics is given in "An Introduction to Statistical Learning" [[3]](#islr), including applications in R. The examples demonstrated in this book implemented in Python can be found in a notebook by J. Warmenhoven [[5]](#islr-python). A more comprehensive coverage can be found in "Elements of Statistical Learning" [[4]](#esl). 


In [1]:
%pylab inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


## Data

We will use a dataset on peer-to-peer lending, given in `loansData.csv` within your repository,  also available at: https://spark-public.s3.amazonaws.com/dataanalysis/loansData.csv 

Let us read the data into a pandas DataFrame and inspect the variables and the contents of the dataset. For this purpose, let's call `.info()` on our pandas DataFrame:

In [2]:
# use the pd.read_csv() method to parse and read the csv data into a pandas DataFrame
# please make sure to name your DataFrame 'loansData'


# use the .info() or .describe() methods to inspec the data columns


In [3]:
# explore the contents of the data by using the .head() method


In [4]:
# count the occurences of unique values of the Home.Ownership column using the .value_counts() method


We can inspect the contents of the dataset and convert some data columns to numeric representation if necessary. While preparing data, look out for these issues:

* Check for NaN entries, before and after data preparation

* Numeric data can be represented and interpreted as strings (due to appended measurement units, percentage signs, etc.)

* Incorrect data (if we have the luxury of knowing something about our variables, we can detect values which our out of range, or incorrect in another way)

In [5]:
#use the dropna() method to drop any NA or NaN entries (rows, not columns!)

#by using lambda abstraction (lambda x: f(x)) and the pandas DataFrame .apply() method remove the '%' signs from the Interest.Rate variable

#get dummy variables for Home.Ownership and add them to the DataFrame
#use the .astype('category') method to represent the column as categorical data
#use pd.get_dummies() to get the dummy variables, and pd.concat() to add them to our DataFrame


# loansData['Debt.To.Income.Ratio'] = loansData['Debt.To.Income.Ratio'].apply(lambda x: x.replace('%','')).astype(float)

# loansData['Loan.Length'] = loansData['Loan.Length'].apply(lambda x: x.replace(' months','')).astype(float)

# loansData['FICO'] = loansData['FICO.Range'].apply(lambda x: x.split('-')[1]).astype(float)

# loansData = loansData.dropna()

# loansData.head(10)

Check the variables in our dataset:

In [6]:
#use .info() to check the contents of the processed data


Divide the data into train and test datasets:

In [7]:
from sklearn.cross_validation import train_test_split

#use the train_test_split function to split the dataset into training and test, with test_size = 0.25


Let us select all the numeric variables, and the dummy variables created from the Home.Ownership categorical variable and ignore (for now) other categorical variables.

Before performing any sort of linear regression the data needs to be checked for (multi)collinearities. Let us check the condition number of the correlation matrix and remove potentially colinear variables.

In [8]:
# all the considered input variables are given here
input_variables = ['Amount.Requested',
                     'Amount.Funded.By.Investors',
                     'Loan.Length',
                     'Debt.To.Income.Ratio',
                     'Monthly.Income',
                     'Open.CREDIT.Lines',
                     'Revolving.CREDIT.Balance',
                     'Inquiries.in.the.Last.6.Months',
                     'FICO',
                     'MORTGAGE',
                     'OTHER',
                     'OWN',
                     'RENT']

# calculate the condition number of the correlation matrix of the input variables
# use np.linalg.cond() and the pandas .corr() methods

# inspect the correlation matrix and identify collinearities

#define the new input variable list (without the dropped ones) and calculate the condition number of the correlation


## Linear Regression

We will use linear regression to answer some important questions which might arise in a real-life setting:
* Is there a relationship between the borrower information and the interest rate?
* How strong is the relationship?
* Which variables influence the interest rate and how string is the effect?
* Can we predict (and with what accuracy) the loan interest rate by knowing the information on the borrower?

The linear regression model assumes a linear relationship between the input vector $X = [X_1,X_2,...,X_p]^T$ and the output $Y$: 

$$Y = \beta_0 + \sum_{j = 1}^{p}\beta_jX_j + \epsilon.$$

Here $\beta_0$ is the intercept term - the expected value of $Y$ when $X = 0$, and $\beta_j$ is the slope - the average increase in $Y$ associated with a one-unit increase in $X_j$. The error term $\epsilon$ captures everything the model cannot predict: the true relationship may not be linear, there may be other variables unaccounted for, or there may be measurement or other variation in the data.

Typically the coefficients $\beta = [\beta_0,\beta_1,...,\beta_p]^T$ are estimated to minimize the residual sum of squares (RSS):

$$RSS(\beta) = \sum_{i = 1}^{N}(y_i - \hat{y}_i)^2 = (\mathbf{y}-\mathbf{X}\beta)^T(\mathbf{y}-\mathbf{X}\beta),$$

where $\mathbf{X} \in \mathbb{R}^{N\times (p+1)}$ contains the input vectors (with 1 in the first position for the intercept term), and $\mathbf{y} \in \mathbb{R}^{N}$ is the vector of outputs in the training set.

Assuming that $\mathbf{X}$ has full column rank ($\mathbf{X}^T\mathbf{X}$ is positive definite), the least squares estimate of the coefficients reads:

$$\hat{\beta} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$$

The linear regression coefficients can be estimated by using the `linear_model.LinearRegression` class from `scikit-learn` and the `fit()` method. The regression coefficients $\beta$ can be accessed in the `intercept_` and `coef_` attributes. The predicted values for a set of inputs can be obtained using the `predict()` method. 

Detailed documentation on these, along with a few useful examples, can be found at:  http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

Let us try out the univariate linear regression: select an input variable, initialize the model, fit the model to the data and inspect the coefficients.

In [9]:
from sklearn import linear_model

# use the Debt.To.Income.Ratio as the input variable and fit a linear regression model
# use the linear_model.LinearRegression class and the .fit() method

# access and print the regression coefficients, using the intercept_ and coef_ attributes


Plot the training data and the regression line, using the fitted model predictions. Model predictions are obtained using the .predict() method.

In [10]:
# use .scatter() in matplotlib.pyplot to plot the data and .plot() to add the regression line
# to obtain the regression line, you can use the model predictions from the LinearRegression.predict() method


Other simple things to explore:
* Calculate and plot residuals
* Inspect the residual distribution (probability plot - implemented in scipy.stats.probplot http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.probplot.html)


### Coefficient Estimates

#### t-test
The t-test is used to check the significance of individual regression coefficients. The null hypothesis to test the significance of a particular regression coefficient estimate $\hat{\beta_j}$ is $H_0: \beta_j = 0$, and the alternative $H_1: \beta_j \neq 0$. The test statistic is based on the $t$-distribution and is given by:

$$t = \frac{\beta_j - 0}{SE(\beta_j)},$$

where $SE(\beta_j)$ is the standard error of the $\beta_j$ estimate. The t-statistic basically measures the number of standard deviations that $\beta_j$ is away from 0. If there is no relationship between $X_j$ and $Y$, the t-statistic will have a t-distribution with $N-p-1$ degrees of freedom. In most cases we test whether $\beta_0 = 0$ (these t-statistics are used to test the significance of the corresponding regressor) - however, when the t-statistic is needed to test the hypothesis of the form $H_0: \beta = \beta_0$, then a non-zero $\beta_0$ may be used. The probability of observing any value equal or greater than $|t|$ is the p-value.

The scikit-learn module does not calculate this information, but there are other ways to retrieve it:
* write a piece of code to calculate it when necessary
* extend the LinearRegression class to do this
* use another package, like scipy or statsmodels

Let us use the statmodels implementation of the ordinary least squares to perform the regression and inspect the significance of the coefficient estimates. The documentation can be found here:
http://statsmodels.sourceforge.net/devel/generated/statsmodels.regression.linear_model.OLS.html

In [11]:
import statsmodels.api as sm

#use the statsmodels class OLS and the .fit() method
#statsmodels does not automatically include an intercept term in the linear regression model
#use sm.add_constant() to append a constant term


### Model accuracy
We can measure the model accuracy by using several different methods:

#### RSE
The residual standard error (RSE) is an estimate of the standard deviation, and is given by:

$$RSE = \sqrt{\frac{1}{N-p-1}RSS}$$

We can calculate the percentage error of the model as the quotient of the RSE by the output mean value: $\frac{RSE}{\hat{Y}}$.

Note that due to the fact that $RSE$ accounts for $p$, models with more variables can have higher RSE if the decrease in RSS is small relative to the increase in $p$.

#### $R^2$ 
The $R^2$ measure, as opposed to the RSE, provides a measure of the proportion of variance explained by the model. It always takes on a value between 0 and 1 and is given by:

$$R^2 = 1 - \frac{RSS}{TSS},$$

where $TSS = \sum_{i = 1}^{N}(y_i - \bar{y})^2$ is the total sum of squares. 



Let us inspect the accuracy of our univariate regression model, use the .score() method to calculate the $R^2$ value, for both training and test data.

In [12]:
#use the .predict() method to calculate residuals


#calculate the RSS, use np.sum()


#calculate the RSE, using np.sqrt() and len(X) (for the number of observations)


#using RSE and the output mean (using np.mean), calculate the percentage error


#using the LinearRegression.score() method, calculate the R^2 score for training and test sets


### Considerations:

* How do you comment the fact that this model has such a good p-value for the coefficient estimate t-test, but a very low $R^2$ score?

* Can the $R^2$ score for test set be better than the one for the training set?

* Can the $R^2$ score be negative?


Let us now fit a **multivariate linear regression** model using multiple input variables. Estimate the model accuracy and compare it to the univariate case. Is this model better? 


A very important tool in assessing multivariate regression models is the F-test.


### F-test
The F-test is used to check the significance of multiple regression coefficients. The null hypothsis reads: $H_0: \beta_1,\beta_1,...\beta_p = 0$ and the alternative is $H_1:$ at least one $\beta_j$ is non-zero. The test statistic is given by:

$$F = \frac{(TSS-RSS)/p}{RSS/(N-p-1)}.$$
 
The F-test is also included in the OLS class within the statsmodels package.

In [13]:
# just a reminder on our input_variables
input_variables = ['Amount.Requested',
                     'Loan.Length',
                     'Debt.To.Income.Ratio',
                     'Monthly.Income',
                     'Open.CREDIT.Lines',
                     'Revolving.CREDIT.Balance',
                     'Inquiries.in.the.Last.6.Months',
                     'FICO',
                     'MORTGAGE',
                     'OTHER',
                     'OWN']

#use the LinearRegression class to fit a multivariate linear regression model

# calculate the R^2 score for both the training and test sets


Including additional predictions seems to have improved the model - let us check the F-statistic:

In [14]:
# use the statsmodels OLS class to fit the multivariate linear regression model and calculate the F-statistic
# don't forget to add a constant term to the data


Obviously the model has improved by including additional input variables. However, we are almost certain that some of these are more useful for prediction than the others. The scikit-learn package includes several procedures for feature selection. Let us use the SelectKBest function to uncover $K$ individually most significant features. The documentation can be found at: http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html

In [15]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

#use the SelectKBest() class to select a couple of most important regression variables and print the selected column names


#use the .get_support() method to access the mask (boolean indices) of the selected features


#fit the reduced model and calculate the R^2 score



### Exercise:

* Use multivariate linear regression to explore possible interactions between variables of the dataset, such as products or quotients.

* Include basis expansions of the input variables in the regression - maybe some of the relationships are not linear?



## Regularized regression models

The regularized least squares methods are used to either avoid ill-posed least squares problems when the number of variables is greater than the number of observations, or to avoid overfitting and aid the generalization power of the model. All of the regularization techniques used below include minimizing a linear combination of the RSS and the estimated coefficient magnitudes:
* **Ridge regression** uses L2 norm of the coefficient magnitudes
* **Lasso regression** uses L1 norm of the coefficient magnitudes
* **Elasticnet** uses a combination of the L1 and L2 norms

Since all of these methods include coefficient magnitudes in the penalty function, the parameters and consequently the results will be considerably dependent on the magnitudes of the input variables. This is why it is always recommended to adjust the input variable magnitues by standardizing or normalizing the input.


### Ridge regression
By adding the L2 norm of the coefficient magnitudes to the minimization function, ridge regression avoids overfitting while keeping all of the predictor variables included. The penalty function reads:

$$\underset{\beta}{min\,} {{|| \beta X  - y||_2^2} + \alpha {||\beta||_2^2}}.$$

Due to the added term, with increased value of $\alpha$ the coefficient magnitudes will decrease. Basicall, the model will rely less on a small set of large predictors (due to large corresponding coefficients) and will tend to distribute these weights accors all variables - thus reducing the possibility of overfitting due to sampling flaws.

Ridge regression can be estimated using the sklearn.linear_model.Ridge class the .fit() method. The documentation can be found at: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html


### Lasso regression
Instead of the L2 norm of the coefficient magnitudes, lasso regression uses the L1 prior as a regularizer. The objective function to minimize is:

$$\underset{\beta}{min\,} { \frac{1}{2N} ||\beta X - y||_2 ^ 2 + \alpha ||\beta||_1}.$$

Due to the L1 norm, instead of evenly distributing the coefficients among all the predictor variables, Lasso results in sparse estimates of the regression coefficients - the model tends to set most coefficients to zero, and the other will be assigned high magnitues. Due to this, and inheret variable selection is performed. This is especially useful in cases when the number of variables is larger than the number of observations and the ordinary least squares problem is ill-posed.

Lasso regression is given in the sklearn.linear_model.Lasso class and its parameters can be estimated by calling the .fit() method. The documentation can be found at: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html  


### ElasticNet
The ElasticNet regression model is trained with both L1 and L2 as regularizers. This combination allows for learning a sparse model where few of the weights are non-zero like Lasso, while still maintaining the regularization properties of Ridge.

Elastic-net is useful when there are multiple correlated features - Lasso is likely to pick one of these at random, while elastic-net is likely to pick both. The objective function to minimize is in this case:

$$\underset{\beta}{min\,} { \frac{1}{2N} ||\beta X - y||_2 ^ 2 + \alpha \rho ||\beta||_1 +
\frac{\alpha(1-\rho)}{2} ||\beta||_2 ^ 2}.$$

ElasticNet regression is given in the sklearn.linear_model.ElasticNet class and its parameters can be estimated by calling the .fit() method. The documentation can be found at: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html


<br>


Let us import the necessary modules:

In [16]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

Let us emulate a situation where the number of variables is very high compared to the number of observations in the training set. We will include all of the categorical variables previously ignored.

In [17]:
#first include all the previously ignored predictor variables - the dummy variables for State and Loan.Purpose

# loansData.loc[:,'State'] = loansData.loc[:,'State'].astype('category')
# dummies = pd.get_dummies(loansData['State'])
# loansData = pd.concat([loansData, dummies], axis=1)

# loansData.loc[:,'Loan.Purpose'] = loansData.loc[:,'Loan.Purpose'].astype('category')
# dummies = pd.get_dummies(loansData['Loan.Purpose'])
# loansData = pd.concat([loansData, dummies], axis=1)


In [18]:
#let us split the data into training and test sets, in such a way that N < p

# train, test = train_test_split(loansData, test_size = 0.98,random_state=1950)
# input_variables = train.select_dtypes(include=['float64','int64']).drop('Interest.Rate', axis=1).columns

# X = train[input_variables]
# Y = train['Interest.Rate']

# print train.shape
# print test.shape

### Ridge trace:
Use the sklearn.linear_model regularized regressions to explore the relationship between the regularization parameter $\alpha$ and the estimated coefficient values for ridge regression.

In [19]:
# alphas = np.linspace(0.001, 5, num=1000)
# ridge_regr = Ridge(alpha = alphas[0],normalize=True)

# coefficients = []
# train_performance = []
# test_performance = []
# for a in alphas:
#     ridge_regr.set_params(alpha=a)
#     ridge_regr.fit(X,Y)
#     coefficients.append(ridge_regr.coef_)
#     train_performance.append(ridge_regr.score(X, Y))
#     test_performance.append(ridge_regr.score(test[input_variables], test['Interest.Rate']))



# figure(num=None, figsize=(15, 6), dpi=200)
# h = plt.plot(alphas, coefficients)
# plt.xlabel('alpha')
# plt.ylabel('coefficients')
# plt.grid(True)
# plt.show()

# figure(num=None, figsize=(10, 6), dpi=200)
# plt.subplot(2, 1, 1)
# plt.semilogx(alphas, train_performance, label='Train')
# plt.semilogx(alphas, test_performance, label='Test')
# plt.legend(loc='lower left')
# plt.ylim([0, 1.2])
# plt.xlabel('Regularization parameter')
# plt.ylabel('Performance')
# plt.show()

Now repeat this procedure and inspect the relationship between the parameter $\alpha$ values and the Lasso regression coefficients. Also explore the train and test set model performance.

In [20]:
#copy-paste the code in the previous cell and use the Lasso() class to fit a lasso regularized regression model


Do the same for the values of the $\alpha$ and $\rho$ parameters and the ElasticNet regression:

In [21]:
#copy-paste the code in the previous cell and use the ElasticNet() class to fit a lasso regularized regression model
#set the value of the l1_ratio to 0.9


### Choosing $\alpha$

The parameter $\alpha$ is usually chosen using K-fold cross validation: we pick the $\alpha$ with the least overall cross validation error (CVE). This is implemented in scikit-learn within the RidgeCV, LassoCV and ElasticNetCV classes.

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV

In [22]:
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV

### Exercise:

* **Fit a multivariate linear regression model (non-regularized) and inspect the model accuracy**
    What is the $R^2$ score on the train data? What is the score on the test data? How do you comment this?


* **Fit an L2-regularized regression model (Ridge regression) to the dataset and inspect the model accuracy**
    Find a favorable value of the parameter $\alpha$ (normalize the input variables). What are the $R^2$ scores for the train and test data?
    
    
* **Fit an L1-regularized regression model (Lasso regression) to the dataset and inspect the model accuracy**
    Which variables have non-zero coefficients? Do they match the selected coefficients in the earlier example?
    
    
* **Fit an ElasticNet regression model to the dataset and inspect the model accuracy**
    What is the accuracy of the model on the train and test datasets?


### Variable selection using Lasso

Let us turn back to the Lasso L1 regularized regression model. Due to the inherent sparsity of the coefficient estimates, this is an especially convenient method for variable selection. The idea is simple: find an optimal value of $\alpha$ using cross-validation, and then explore the estimated regression coefficients - those different than 0 correspond to valuable input variables.


Let us detect which coefficients our Lasso model chose as significant.

In [23]:
#use the fitted lasso model and the .coef_ attribute to access the coefficients
#print out those column namse whose corresponding coefficients are != 0



How do these coefficients compare to our identified significant coefficients above (when using SelectKBest)? Note that these are completely different models, due to added variables and a significantly reduced training set - nevertheless, some variables seem to be important in both cases.




# Logistic regression

Rather than predicting the interest rate given a set of input parameter values, sometimes it is useful to consider binary outcomes: "Is it possible to get a loan with an interest rate of 12%, given one's parameter values?"

If we characterize all loans with the interest rate above the threshold of 12% as unaffordable and all others as affordable - the question we face read: "What is the probability of getting an affordable loan, given one's parameters?"

### Classification via logistic regression
Logistic regression is a linear model for classification, also known in the literature as logit regression, maximum-entropy classification (MaxEnt) or the log-linear classifier. In this model, the probabilities describing the possible outcomes of a single trial are modeled using a logistic function.

The implementation of logistic regression in scikit-learn includes optional L2 or L1 regularization. As in previous examples, regularization helps avoid overfitting and deals with a large number of variables with insufficient observations. As an optimization problem, binary class L2 penalized logistic regression minimizes the following cost function:
$$\underset{w, c}{min\,} \frac{1}{2}w^T w + C \sum_{i=1}^n \log(\exp(- y_i (X_i^T w + c)) + 1) .$$
Similarly, L1 regularized logistic regression solves the following optimization problem
$$\underset{w, c}{min\,} \|w\|_1 + C \sum_{i=1}^n \log(\exp(- y_i (X_i^T w + c)) + 1) .$$

The parameter $C$ can be set to very high values to simply annul the regularization. More can be found in the documentation: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

A very useful notebook covering the logistic regression model in statsmodels is given here: http://blog.yhat.com/posts/logistic-regression-and-python.html

Another notebook, using scikit-learn is available here: http://www.dataschool.io/logistic-regression-in-python-using-scikit-learn/

Literature
-----------

<a name="scikit-linear">[1]</a> scikit-learn documentation - 1.1. Generalized Linear Models, [link](http://scikit-learn.org/stable/modules/linear_model.html)

<a name="statsmodels">[2]</a> statsmodels documentation, [link](https://pypi.python.org/pypi/statsmodels)

<a name="islr">[3]</a> James, G., Witten, D., Hastie, T., and Tibshirani, R. (2014), An Introduction to Statistical Learning: With Applications in R. Springer Publishing Company, Inc., [link](http://www-bcf.usc.edu/~gareth/ISL/)

<a name="esl">[4]</a> Hastie, T.; Tibshirani, R. & Friedman, J. (2001), The Elements of Statistical Learning, Springer New York Inc., New York, NY, USA. [link](http://statweb.stanford.edu/~tibs/ElemStatLearn/)

<a name="scikit-linreg">[5]</a> ISLR - Python, [link](https://github.com/JWarmenhoven/ISLR-python)